# Intune Module Setup Example Notebook

This notebook creates 1 table (devices) into a two new Spark databases called s2p_intune and s2np_intune. 

Attach this notebook to your configured spark pool, and click "Run all" for the initial module setup (outlined in the tutorial).

### Provision storage accounts

The storage account variable has to be changed to the name of the storage account associated with your Azure resource group.

In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, TimestampType
from pyspark.sql.functions import *
from pyspark.sql.window import Window


# data lake and container information
storage_account = 'stoeadaihackathon'
use_test_env = False

if use_test_env:
    stage1np = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage1np'
    stage2np = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2np'
    stage2p = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2p'
else:
    stage1np = 'abfss://stage1np@' + storage_account + '.dfs.core.windows.net'
    stage2np = 'abfss://stage2np@' + storage_account + '.dfs.core.windows.net'
    stage2p = 'abfss://stage2p@' + storage_account + '.dfs.core.windows.net'

StatementMeta(medium1, 7, 1, Finished, Available)

### Load Raw Data from Lake
To ensure that that the right tables are loaded, confirm that the file paths match your data lake storage containers. 

Make sure to reference either the tutorial guide or the note below, as the current notebook setup for this module reads *.csv rather than a specific CSV file.

In [2]:
# load needed device table from the CSV in the data lake storage
# Note: you can change the "*.csv" to the specific devices CSV file name (which is generated as the date and time of the pipeline trigger)
dfIntuneDevices = spark.read.format('csv').load(f'{stage1np}/intune/devices/*.csv', header='true')

StatementMeta(medium1, 7, 2, Finished, Available)

## 1. Devices table
Contains all devices (from students and teachers) at a school-system level

** Databases and tables used: **

 - None

**CSV files used:**

- the file from: stage1np/intune/devices/*.csv

**Database and table created:**

1. Spark DB: s2p_intune
- Table: devices
2. Spark DB: s2np_intune
- Table: devices

In [3]:
display(dfIntuneDevices.limit(10))

StatementMeta(medium1, 7, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0fd3e91c-73ee-4543-8ff9-6a3dcfbe34ce)

### Rename the column names to be get rid of hyphens and spaces
Uncomment the cell below if you have triggered the pipeline and wish to ingest your own data, rather than the test data.

In [ ]:
#dfIntuneDevices = dfIntuneDevices.withColumnRenamed('Device name', 'DeviceName').withColumnRenamed('Managed by', 'ManagedBy')
#dfIntuneDevices = dfIntuneDevices.withColumnRenamed('OS version', 'OSVersion').withColumnRenamed('Last check-in', 'LastCheckIn')
#dfIntuneDevices = dfIntuneDevices.withColumnRenamed('Primary user UPN', 'PrimaryUserUPN').withColumnRenamed('Device ID', 'DeviceID')

### Add an additional column "accessOutsideOfSchool"

This "accessOutsideOfSchool" column uses the "lastCheckIn" column information to determine if a specific student's device has access outside of school, based on the conditions:

 - If the last check in was on a weekend (i.e. Saturday or Sunday), then "accessOutsideOfSchool" is true.
 - If the last check in was before 9 AM (9:00) on a weekday, then "accessOutsideOfSchool" is true.
 - If the last check in was after 4 PM (16:00) on a weekday, then "accessOutsideOfSchool" is true. 

Otherwise, "accessOutsideOfSchool" is defaulted to false. 

Also this code creates a "lastCheckInDate" date-type column, used for the date filter in the PowerBI dashboard.

In [18]:
dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInTime', split(col('lastCheckIn'), ' ').getItem(1))
dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInDate', split(col('lastCheckIn'), ' ').getItem(0))
dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInHourOfDay', split(col('lastCheckInTime'), ':').getItem(0))
dfIntuneDevices = dfIntuneDevices.drop('lastCheckInTime')

dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInDate', to_date(col('lastCheckInDate'), 'yyyy-MM-dd'))
dfIntuneDevices = dfIntuneDevices.withColumn('lastCheckInDayOfWeek', date_format(col('lastCheckIn'), "E"))

dfIntuneDevices = dfIntuneDevices.withColumn('AccessOutsideOfSchool', when(col('lastCheckInDayOfWeek') == "Sat", "true").otherwise(when(col('lastCheckInDayOfWeek') == "Sun", "true").otherwise(when(col('lastCheckInHourOfDay') >= 16, "true").otherwise(when(col('lastCheckInHourOfDay') < 9, "true").otherwise("false")))))
# Can comment out this drop if you don't want to drop these two columns
dfIntuneDevices = dfIntuneDevices.drop('lastCheckInDayOfWeek').drop('lastCheckInHourOfDay')

display(dfIntuneDevices.limit(10))

StatementMeta(medium1, 7, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 26a06c19-36fe-4dba-b35a-ea482711c51c)

## Write Data Back to Lake

### Writing to Stage 2np

In [19]:
# write back to the lake in stage 2 ds2_main directory
dfIntuneDevices.coalesce(1).write.format('parquet').mode('overwrite').save(stage2np + '/intune/devices')

StatementMeta(medium1, 7, 19, Finished, Available)

### Writing to Stage 2p
Pseudonymizing the primaryUserUPNs (userPrincipalNames) data from the devices CSV.

In [20]:
%run /OEA_py

StatementMeta(, 7, -1, Finished, Available)

In [21]:
oea = OEA()

StatementMeta(medium1, 7, 21, Finished, Available)

2021-10-19 15:55:51,160 - OEA - DEBUG - OEA initialized.
OEA initialized.

In [22]:
devicesSchema = [['DeviceName', 'string', 'no-op'],
                            ['ManagedBy', 'string', 'no-op'],
                            ['Ownership', 'string', 'no-op'],
                            ['Compliance','string','no-op'],
                            ['OS', 'string', 'no-op'],
                            ['OSVersion', 'string', 'no-op'],
                            ['LastCheckIn', 'timestamp', 'no-op'],
                            ['PrimaryUserUPN', 'string', 'hash'],
                            ['DeviceID', 'string', 'no-op'],
                            ['AccessOutsideOfSchool', 'boolean', 'no-op']]

df_pseudo, df_lookup = oea.pseudonymize(dfIntuneDevices, devicesSchema)

df_pseudo.coalesce(1).write.format('parquet').mode('overwrite').save(stage2p + '/intune/devices')

StatementMeta(medium1, 7, 22, Finished, Available)

### Load to Spark DB

In [23]:
# Create spark db to allow for access to the data in the delta-lake via SQL on-demand.
# This is only creating metadata for SQL on-demand, pointing to the data in the delta-lake.
# This also makes it possible to connect in Power BI via the azure sql data source connector.
def create_spark_db(db_name, source_path):
    spark.sql(f'CREATE DATABASE IF NOT EXISTS {db_name}')
    spark.sql(f"DROP TABLE IF EXISTS {db_name}.devices")
    spark.sql(f"create table if not exists {db_name}.devices using PARQUET location '{source_path}'")
    
create_spark_db('s2np_intune', stage2np + '/intune/devices')
create_spark_db('s2p_intune', stage2p + '/intune/devices')

StatementMeta(medium1, 7, 23, Finished, Available)

## Reset Data Processing

In [ ]:
# Uncomment line 8 if you would like to walk through the process again from the beginning
def reset_all_processing():
    oea.rm_if_exists(stage2np + '/intune/devices')
    oea.rm_if_exists(stage2p + '/intune/devices')
    oea.drop_db('s2np_intune')
    oea.drop_db('s2p_intune')

#reset_all_processing()